In [1]:
import face_recognition
import os


from PIL import Image, ImageDraw
import numpy as np


import pymysql

# MySQL 연결 - DB server에 연결, 정상적으로 연결이 수립되면 커넥션 객체를 반환
#conn = pymysql.connect(host='localhost', user='root', password='1234', db='encore', charset='utf8')
conn = pymysql.connect(host='localhost', user='sptest', password='sptest', db='testdb', charset='utf8')
# 커서 생성 - 커넥션 객체를 통해 데이터베이스에서 SQL문을 대신 실행해주고 결과를 반환해 줄 커서 객체를 생성한다.
cur = conn.cursor()
# 데이터 조회 - 커서에 SELECT로 조회한 결과를 한꺼번에 저장
# sql = 'select * from departments where department_id=%d'
# vals = (department_id,)
# cur.execute(sql, vals)
sql = "select * from USER"
cur.execute(sql)

# 조회결과를 커서에서 fetch - 조회한 데이터를 커서객체에서 한 행씩 접근하여 출력
#여러 줄 출력
for row in cur:
		print(row[0], row[1])

#한 줄 출력
# row = cur.fetchone()
# if row == None:
#     print('검색결과없음')
# else:
#     print(row[0], row[1], row[2], row[3])


##### MySQL에 이미지 저장
import base64
import io

jukyuknam nam
yoojungkim kim
suzibae bae


In [2]:
# 얼굴인식한 사람이 입력한 아이디를 가져옴
sql = "select * from FACE"
cur.execute(sql)
row = cur.fetchone()
unknownId = row[0]
print(unknownId)

yoojungkim


In [3]:
# 이미 db에 저장되어있는 사진과
# 얼굴인식한 사람이 입력한 아이디의 사진과 비교
# 이미 db에 저장되어있는 사진들은 각각 encoding이 완료된 상태여야 함***
userId_dict = {"jukyuknam":"person_1", "yoojungkim":"person_2", "suzibae":"person_3", "bogumpark":"person_4"}
person_to_compare = userId_dict[unknownId]
print(person_to_compare)

# The training data would be all the face encodings from all the known images and the labels are their names
encodings = []
names = []

# Loop through each person in the training directory
pix = os.listdir("train_dir/" + person_to_compare)

# Loop through each training image for the current person
for person_img in pix:
    # Get the face encodings for the face in each image file
    face = face_recognition.load_image_file("train_dir/" + person_to_compare + "/" + person_img)
    face_bounding_boxes = face_recognition.face_locations(face)

    #If training image contains exactly one face
    if len(face_bounding_boxes) == 1:
        face_enc = face_recognition.face_encodings(face)[0]
        # Add face encoding for current image with corresponding label (name) to the training data
        encodings.append(face_enc)
        names.append(person_to_compare)
    else:
        print("train_dir/" + person_img + " was skipped and can't be used for training")
            
            
print(names)

person_2
['person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2', 'person_2']


In [5]:
#얼굴인식한 사진을 mysql에서 갖고 옴
def retriveImage(userId):
    sql = "SELECT userFace FROM FACE WHERE userId=%s"
    cur.execute(sql, (userId,))
    results = cur.fetchall()
    # the returned data will be a list of list
    image = results[0][0]
    # Decode the string
    binary_data = base64.b64decode(image)
    # Convert the bytes into a PIL image
    image = Image.open(io.BytesIO(binary_data))
    image.type
    # Display the image using default image app
    #image.show()
    return image

unknown_Image = retriveImage(unknownId)

#convert a PIL Image into a NumPy array
unknown_Image2 = np.array(unknown_Image)


# 얼굴인식 사진을 person_to_compare로 인코딩한 사진들과 비교
def face_recognition_result(image_to_test):
    image_to_test_encoding = face_recognition.face_encodings(image_to_test)[0]

    # See how far apart the test image is from the known faces
    face_distances = face_recognition.face_distance(encodings, image_to_test_encoding)

    total=0

    for i, face_distance in enumerate(face_distances):
        # print("The test image has a distance of {:.2} from known image #{}".format(face_distance, i))
        total += face_distance
        # print()

    print(total/20)
    print(1 - total/20)
    
    if (total/20 < 0.40):
        print(unknownId + " TRUE")
    else:
        print(unknownId + " FALSE")

face_recognition_result(unknown_Image2)

AttributeError: 'JpegImageFile' object has no attribute 'type'